In [ ]:
# default_exp kubernetes_runner

In [22]:
#export
import datetime
import jinja2
import errno
import os
import os.path
import tempfile
import gcp_runner.core

from gcp_runner import ai_platform_constants
from gcp_runner.core import build_and_push_docker_image, run_process, get_run_python_args, format_job_dir, print_tensorboard_command

def _get_not_none(arg, default):
    return default if arg is None else arg

def _find_project_file(file_name):
    searched_dirs = []
    # Look in current directory when running from notebook
    template_path = os.path.join(os.getcwd(), file_name)
    if os.path.isfile(template_path):
        return template_path
    searched_dirs.append(template_path)
    
    # Look in package directory when running pip installed package
    template_path = os.path.join(os.path.dirname(gcp_runner.core.__file__), file_name)
    if os.path.isfile(template_path):
        return template_path
    searched_dirs.append(template_path)

    # Look in package parent directory when running editable pip installed package
    template_path = os.path.join(os.path.dirname(os.path.dirname(gcp_runner.core.__file__)), file_name)
    if os.path.isfile(template_path):
        return template_path
    searched_dirs.append(template_path)
    error_message = 'niether %s can be found' % searched_dirs.join(',')
    raise FileNotFoundError(
        errno.ENOENT, error_message, file_name)
        
def run_docker_image(
    func, 
    job_dir,
    image_uri,
    build_docker_file=None,
    dry_run=False,
    job_name=None,
    region=None,
    port=5000,
    privileged=False,
    master_machine_type: ai_platform_constants.MachineType = None,
    master_accelerator_count=None,
    parameter_machine_type: ai_platform_constants.MachineType = None,
    parameter_machine_count=None,
    parameter_accelerator_count=None,
    worker_machine_type: ai_platform_constants.MachineType = None,
    worker_machine_count=None,
    work_accelerator_count=None,
    tpu_count=None,
    distribution_strategy_type: ai_platform_constants.DistributionStrategyType = None,
    use_distribution_strategy_scope: bool = None,
    **kwargs):
    
    if not image_uri:
        raise ValueError('image_uri argument is required')
        
    if not job_name:
        job_name = 'kubernetes-runner-train-docker'
        
    if build_docker_file is not None:
        result = build_and_push_docker_image(build_docker_file, image_uri, dry_run=dry_run)
        if result:
            return result
        
    job_dir = format_job_dir(job_dir)
    print_tensorboard_command(job_dir)

    master_machine_count = 1
    if distribution_strategy_type == ai_platform_constants.DistributionStrategyType.MIRRORED_STRATEGY:
        master_accelerator_count = _get_not_none(master_accelerator_count, 2)
    elif distribution_strategy_type == ai_platform_constants.DistributionStrategyType.CENTRAL_STORAGE_STRATEGY:
        master_accelerator_count = _get_not_none(master_accelerator_count, 2)
    elif distribution_strategy_type == ai_platform_constants.DistributionStrategyType.PARAMETER_SERVERSTRATEGY:
        parameter_machine_count = _get_not_none(parameter_machine_count, 1)
        worker_machine_count = _get_not_none(worker_machine_count, 2)
    elif distribution_strategy_type == ai_platform_constants.DistributionStrategyType.MULTI_WORKER_MIRRORED_STRATEGY:
        master_accelerator_count = _get_not_none(master_accelerator_count, 2)
        work_accelerator_count = _get_not_none(work_accelerator_count, 2)
        worker_machine_count = _get_not_none(worker_machine_count, 2)
    elif distribution_strategy_type == ai_platform_constants.DistributionStrategyType.TPU_STRATEGY:
        # minimal available number for central1-c  # https://cloud.google.com/tpu/docs/types-zones
        tpu_count = _get_not_none(tpu_count, 8)
    elif distribution_strategy_type == ai_platform_constants.DistributionStrategyType.ONE_DEVICE_STRATEGY:
        pass
    
    master_machine_type = _get_not_none(master_machine_type, ai_platform_constants.MachineType.N1_STANDARD_4)
    worker_machine_type = _get_not_none(worker_machine_type, ai_platform_constants.MachineType.N1_STANDARD_4)
    parameter_machine_type = _get_not_none(parameter_machine_type, ai_platform_constants.MachineType.N1_STANDARD_4)
    
    master_accelerator_count = _get_not_none(master_accelerator_count, 0)
    work_accelerator_count = _get_not_none(work_accelerator_count, 0)
    parameter_accelerator_count = _get_not_none(parameter_accelerator_count, 0)

    parameter_machine_count = _get_not_none(parameter_machine_count, 0)
    worker_machine_count = _get_not_none(worker_machine_count, 0)
    tpu_count = _get_not_none(tpu_count, 0)
        
    #TODO: map machine_type to memory and cpu availability, and pass them to template
    
    run_python_args = get_run_python_args(func, **kwargs)
    run_python_args.append("--job-dir=%s" % job_dir)
    if distribution_strategy_type is not None:
        run_python_args.append("--distribution-strategy-type=%s" % distribution_strategy_type)
        if use_distribution_strategy_scope:
            run_python_args.append("--use-distribution-strategy-scope")

    #TODO: properly pass args to python
    yaml = None
    with open(_find_project_file("template.yaml.jinja"), "r") as f:
        print('Generating kubernetes YAML file:')
        yaml = (jinja2.Template(f.read()).render(
            name=job_name,
            image_uri=image_uri,
            port=port,
            privileged=privileged,
            master_machine_count=master_machine_count,
            parameter_machine_count=parameter_machine_count,
            worker_machine_count=worker_machine_count,
            master_accelerator_count=master_accelerator_count,
            parameter_accelerator_count=parameter_accelerator_count,
            work_accelerator_count=work_accelerator_count,
            tpu_count=tpu_count,
            train_dir=job_dir,
            cmdline_args=run_python_args))
    if yaml is None:
        return -1
    print(yaml.replace('\n\n','\n'))
    
    print('Deleting all resources on Kubernetes cluster:')
    #command = 'kubectl delete pod,service,PodSecurityPolicy --all'
    command = 'kubectl delete namespace kubernetes-runner-namespace'
    print(command)
    if not dry_run:
        run_process(command.split(' '))
    
    fd, path = tempfile.mkstemp()
    try:
        with os.fdopen(fd, 'w') as tmp:
            # do stuff with temp file
            tmp.write(yaml)
    except Error as e:
        print(e)
        
    print('Creating Kubernetes kubernetes-runner-namespace namespace:')
    #args = ['kubectl', 'delete', 'pod,service,PodSecurityPolicy', '--all']
    command = 'kubectl create namespace kubernetes-runner-namespace'
    print(command)
    if not dry_run:
        result = run_process(command.split(' '))
        if result:
            return result        
        
    print('Creating Kubernetes cluster:')
    command = 'kubectl create -f %s --namespace=kubernetes-runner-namespace' % path
    print(command)
    if not dry_run:
        result = run_process(command.split(' '))
        if result:
            return result
    
    print('See logs at:') # TODO: provide non-google link as well
    print('https://pantheon.corp.google.com/logs/viewer?minLogLevel=0&expandAll=false&'
          'advancedFilter=resource.type%3D%22container%22%0Aresource.labels.namespace_id%3D%22'
          'kubernetes-runner-namespace%22&dateRangeStart=' + datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"))
    print('Polling Kubernetes logs:')
    return run_process([_find_project_file('poll_kubernetes_logs.sh')])  

In [1]:
from gcp_runner import core
core.export_and_reload_all()

Converted 00_core.ipynb.
Converted ai_platform_constants.ipynb.
Converted ai_platform_runner.ipynb.
Converted entry_point.ipynb.
Converted index.ipynb.
Converted kubernetes_runner.ipynb.
Converted local_runner.ipynb.
Converted sample_code_test.ipynb.
